In [1]:
# features: sentence embeddings+noun embedding+adj embeddding
# or named entity embeddings
# column edit distance??

# https://medium.com/huggingface/universal-word-sentence-embeddings-ce48ddc8fc3a

# Wiki Pre Trained with Fasttext https://fasttext.cc/docs/en/english-vectors.html
# Advances in Pre-Training Distributed Word Representations

In [3]:
# import spaCy??
# https://spacy.io/
import io
import json
import spacy
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [4]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    cnt = 0
    for line in fin:
        cnt += 1
        if cnt % 100000 == 0:
            print(cnt)
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = list(map(float, tokens[1:]))
    return data

In [5]:
def load_questions(fname):
    with open(fname) as f:
        questions_list = f.readlines()

    filtered_questions_list = []
    for question in questions_list:
        question_json = json.loads(question)
        question_json.pop('question1')
        question_json.pop('question2')
        filtered_questions_list.append(question_json)
            
    return filtered_questions_list

In [6]:
def load_tables(part_file_name):
    with open('../WikiSQL/data/'+part_file_name+'.tables.jsonl') as f:
        tables_list = f.readlines()
        
    tables = {}
    for table in tables_list:
        table_json = json.loads(table)
        tables[table_json['id']] = table_json
    return tables

In [7]:
def sentence_embedding(sentence, spacy_model, fastText_model):
    # weight more on nouns that are not stop words
    embed_dim = 300
    noun_amplify = 3
    doc = nlp(sentence)
    embed_res = np.zeros((embed_dim,))
    doc_len = len(doc)
    noun_cnt = 0
    for token in doc:
        if token.lemma_ in fastText_model:
            if token.pos_ == 'NOUN' and token.is_stop == False:
                embed_res += noun_amplify*np.array(fastText_model[token.lemma_])
                noun_cnt += 1
            else:
                embed_res += np.array(fastText_model[token.lemma_])
        else:
            embed_res += np.zeros((embed_dim,))
#         print(fastText_model[token.lemma_][-1], embed_res[-1])
    embed_res = np.array(embed_res)/(doc_len+(noun_amplify-1)*noun_cnt)
    return embed_res

In [8]:
def headers_embedding(headers, spacy_model, fastText_model):
    embed_dim = 300
    header_embedding = np.empty((0, embed_dim))
    for col_name in headers:
        col_embed = sentence_embedding(col_name, spacy_model, fastText_model)
        header_embedding = np.vstack((header_embedding, col_embed))
    return header_embedding

In [9]:
def question_Xy(question_embedding, header_embedding, sel_ind, conds_ind):
    question_embed = 600
    question_X = np.empty((0, question_embed))
    question_y = np.empty((0,1))
    for header_ind in range(len(header_embedding)):
        question_X = np.vstack((question_X, np.concatenate((question_embedding, header_embedding[header_ind]))))
        if header_ind == sel_ind or header_ind == conds_ind:
            question_y = np.vstack((question_y, np.array(1)))
        else:
            question_y = np.vstack((question_y, np.array(0)))
    return question_X, question_y

In [10]:
def obtain_Xy(question_list, table_dict, spacy_model, fastText_model):
    question_embed = 600
    aggreation_embed = 300
    question_Xs = np.empty((0, question_embed))
    question_ys = np.empty((0,1))
    aggreation_Xs = np.empty((0, aggreation_embed))
    aggregation_ys = np.empty((0,1))
    for question in question_list:
        question_embedding = sentence_embedding(question['question'], spacy_model, fastText_model)
        header_embedding = headers_embedding(table_dict[question['table_id']]['header'], spacy_model, \
                                           fastText_model)
        
        question_X, question_y = question_Xy(question_embedding, header_embedding, question['sql']['sel'], \
                                                  question['sql']['conds'][0][0])
        question_Xs = np.vstack((question_Xs, question_X))
        question_ys = np.vstack((question_ys, question_y))
        
        aggreation_Xs = np.vstack((aggreation_Xs, question_embedding))
        if question['sql']['agg'] == 5:
            agg = 0
        else:
            agg = 1
        aggregation_ys = np.vstack((aggregation_ys, np.array(agg)))
        
    return question_Xs, question_ys, aggreation_Xs, aggregation_ys

In [13]:
fastText = load_vectors('wiki-news-300d-1M-subword.vec')

100000
200000
300000
400000
500000
600000
700000
800000
900000


In [14]:
nlp = spacy.load("en_core_web_sm")

In [11]:
train_questions = load_questions('vis_train_questions.txt')
test_questions = load_questions('vis_test_questions.txt')

In [12]:
train_tables = load_tables('train')

In [13]:
test_tables = load_tables('test')

In [108]:
column_train_X, column_train_y, agg_train_X, agg_train_y = obtain_Xy(train_questions, train_tables, nlp, fastText)
column_test_X, column_test_y, agg_test_X, agg_test_y = obtain_Xy(test_questions, test_tables, nlp, fastText)

In [131]:
agg_clf = RandomForestClassifier()
agg_clf.fit(agg_train_X,agg_train_y)
agg_y_pred = agg_clf.predict(agg_test_X)
accuracy_score(agg_test_y, agg_y_pred)

/usr/local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.7

In [132]:
agg_clf = MLPClassifier(hidden_layer_sizes=(256,128))
agg_clf.fit(agg_train_X,agg_train_y)
agg_y_pred = agg_clf.predict(agg_test_X)
accuracy_score(agg_test_y, agg_y_pred)

/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.9

In [119]:
agg_train_y.shape

(150, 1)

In [135]:
col_clf = RandomForestClassifier()
col_clf.fit(column_train_X, column_train_y)

# test_questions = questions[50:]
test_tot = len(test_questions)
correct_cnt = 0
for question in test_questions:
    col_test_X, col_test_y, _, _ = obtain_Xy([question], test_tables, nlp, fastText)
    col_y_pred = col_clf.predict(col_test_X)
    acc = accuracy_score(col_test_y, col_y_pred)
    if acc == 1.0:
        correct_cnt+=1
print("accuracy:", correct_cnt/test_tot)
# np.intersect1d(np.where(column_test_y==1)[0], np.where(col_y_pred==1))

/usr/local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


accuracy: 0.2


In [134]:
column_y_pred = col_clf.predict(column_test_X)
accuracy_score(column_test_y, column_y_pred)

0.732620320855615

In [129]:
col_clf = MLPClassifier(hidden_layer_sizes=(512,256,64))
# col_clf = RandomForestClassifier()
col_clf.fit(column_train_X, column_train_y)

# test_questions = questions[50:]
test_tot = len(test_questions)
correct_cnt = 0
for question in test_questions:
    col_test_X, col_test_y, _, _ = obtain_Xy([question], test_tables, nlp, fastText)
    col_y_pred = col_clf.predict(col_test_X)
    acc = accuracy_score(col_test_y, col_y_pred)
    if acc == 1.0:
        correct_cnt+=1
print("accuracy:", correct_cnt/test_tot)
# np.intersect1d(np.where(column_test_y==1)[0], np.where(col_y_pred==1))

/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy: 0.3333333333333333


In [130]:
column_y_pred = col_clf.predict(column_test_X)
accuracy_score(column_test_y, column_y_pred)

0.7379679144385026

In [341]:
np.where(column_test_y==1)[0]

array([3, 4])

In [335]:
np.where(col_y_pred==1)[0]

array([], dtype=int64)

In [317]:
len(np.where(col_y_pred==1)[0])

13

In [331]:
column_test_X, column_test_y, _, _ = obtain_Xy(questions[50:51], tables, nlp, fastText)

In [17]:
tab_id = '2-15418823-1'
train_tables[tab_id]

{'header': ['Position',
  'Team',
  'Points',
  'Played',
  'Drawn',
  'Lost',
  'Against',
  'Difference'],
 'page_title': '1911 in Brazilian football',
 'types': ['real', 'text', 'real', 'real', 'real', 'real', 'real', 'text'],
 'page_id': 15418823,
 'id': '2-15418823-1',
 'section_title': 'Campeonato Paulista',
 'rows': [['1', 'São Paulo Athletic', '15', '9', '1', '1', '15', '8'],
  ['2', 'Americano-SP', '11', '8', '1', '2', '15', '14'],
  ['3', 'Paulistano', '7', '9', '1', '5', '15', '2'],
  ['4', 'Germânia', '6', '9', '0', '6', '26', '- 12'],
  ['5', 'Ypiranga-SP', '5', '9', '1', '6', '28', '- 15']],
 'caption': 'Campeonato Paulista'}

In [59]:
len(test_questions)

30